

---

# **Chapter 51: Platform-Specific Configuration**

---

## **Learning Objectives**

By the end of this chapter, you will be able to:

- Configure Android-specific settings using AndroidManifest.xml and Gradle build files
- Manage iOS-specific configurations through Info.plist and Podfile settings
- Implement platform-specific permissions for camera, location, notifications, and storage
- Optimize release builds using ProGuard/R8 rules for code shrinking and obfuscation
- Resolve common platform-specific build errors and dependency conflicts
- Configure platform-specific themes, icons, and launch screens
- Handle deep linking and URL schemes across Android and iOS

---

## **Prerequisites**

- Completed Chapter 50: Package Ecosystem Guide
- Working knowledge of Android and iOS project structures
- Understanding of XML and property list (plist) formats
- Familiarity with Gradle and CocoaPods build systems
- Access to macOS for iOS-specific sections (optional but recommended)

---

## **51.1 Android Configuration**

Android configuration in Flutter projects involves the `android/` directory, which contains native Android project files. Understanding these is essential for production apps.

### **AndroidManifest.xml Configuration**

The `AndroidManifest.xml` is the heart of your Android application's configuration, defining its components, permissions, and hardware requirements.

```xml
<!-- android/app/src/main/AndroidManifest.xml -->
<manifest xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:tools="http://schemas.android.com/tools"
    package="com.example.myapp">

    <!-- APPLICATION PERMISSIONS -->
    <!-- Internet permission - required for almost all Flutter apps -->
    <uses-permission android:name="android.permission.INTERNET" />
    
    <!-- Network state - useful for checking connectivity -->
    <uses-permission android:name="android.permission.ACCESS_NETWORK_STATE" />
    
    <!-- Camera permission - required for camera access -->
    <uses-permission android:name="android.permission.CAMERA" />
    
    <!-- Storage permissions - for file access -->
    <!-- READ_EXTERNAL_STORAGE deprecated in API 33+, use granular media permissions -->
    <uses-permission android:name="android.permission.READ_EXTERNAL_STORAGE"
        android:maxSdkVersion="32" />
    <uses-permission android:name="android.permission.WRITE_EXTERNAL_STORAGE"
        android:maxSdkVersion="29" />
    
    <!-- Android 13+ (API 33+) granular media permissions -->
    <uses-permission android:name="android.permission.READ_MEDIA_IMAGES" />
    <uses-permission android:name="android.permission.READ_MEDIA_VIDEO" />
    <uses-permission android:name="android.permission.READ_MEDIA_AUDIO" />
    
    <!-- Location permissions -->
    <uses-permission android:name="android.permission.ACCESS_FINE_LOCATION" />
    <uses-permission android:name="android.permission.ACCESS_COARSE_LOCATION" />
    <!-- Background location (requires special approval on Play Store) -->
    <uses-permission android:name="android.permission.ACCESS_BACKGROUND_LOCATION" />
    
    <!-- Biometric permissions -->
    <uses-permission android:name="android.permission.USE_BIOMETRIC" />
    
    <!-- Notification permissions (Android 13+ requires explicit permission) -->
    <uses-permission android:name="android.permission.POST_NOTIFICATIONS" />
    
    <!-- Bluetooth permissions (Android 12+ requires BLUETOOTH_SCAN, CONNECT) -->
    <uses-permission android:name="android.permission.BLUETOOTH" 
        android:maxSdkVersion="30" />
    <uses-permission android:name="android.permission.BLUETOOTH_ADMIN" 
        android:maxSdkVersion="30" />
    <uses-permission android:name="android.permission.BLUETOOTH_SCAN" 
        android:usesPermissionFlags="neverForLocation" />
    <uses-permission android:name="android.permission.BLUETOOTH_CONNECT" />
    
    <!-- Hardware features (optional but recommended) -->
    <uses-feature android:name="android.hardware.camera" android:required="false" />
    <uses-feature android:name="android.hardware.location.gps" android:required="false" />

    <application
        android:label="My App"
        android:name="${applicationName}"
        android:icon="@mipmap/ic_launcher"
        android:roundIcon="@mipmap/ic_launcher_round"
        android:allowBackup="false"
        android:fullBackupContent="false"
        android:networkSecurityConfig="@xml/network_security_config"
        android:usesCleartextTraffic="false"
        android:requestLegacyExternalStorage="false"
        tools:targetApi="33">
        
        <!-- Main Activity -->
        <activity
            android:name=".MainActivity"
            android:exported="true"
            android:launchMode="singleTop"
            android:theme="@style/LaunchTheme"
            android:configChanges="orientation|keyboardHidden|keyboard|screenSize|smallestScreenSize|locale|layoutDirection|fontScale|screenLayout|density|uiMode"
            android:hardwareAccelerated="true"
            android:windowSoftInputMode="adjustResize"
            android:screenOrientation="portrait">
            
            <!-- Intent filter for main entry point -->
            <intent-filter>
                <action android:name="android.intent.action.MAIN" />
                <category android:name="android.intent.category.LAUNCHER" />
            </intent-filter>
            
            <!-- Deep linking configuration -->
            <intent-filter android:autoVerify="true">
                <action android:name="android.intent.action.VIEW" />
                <category android:name="android.intent.category.DEFAULT" />
                <category android:name="android.intent.category.BROWSABLE" />
                <data android:scheme="https" 
                      android:host="example.com" 
                      android:pathPrefix="/app" />
            </intent-filter>
            
            <!-- Custom URL scheme -->
            <intent-filter>
                <action android:name="android.intent.action.VIEW" />
                <category android:name="android.intent.category.DEFAULT" />
                <category android:name="android.intent.category.BROWSABLE" />
                <data android:scheme="myapp" android:host="open" />
            </intent-filter>
            
            <!-- Specifies an Android theme to apply to this Activity -->
            <meta-data
                android:name="io.flutter.embedding.android.NormalTheme"
                android:resource="@style/NormalTheme" />
                
            <!-- Splash screen drawable -->
            <meta-data
                android:name="io.flutter.embedding.android.SplashScreenDrawable"
                android:resource="@drawable/launch_background" />
        </activity>
        
        <!-- Firebase Cloud Messaging (if using) -->
        <service
            android:name="com.google.firebase.messaging.FirebaseMessagingService"
            android:exported="false">
            <intent-filter>
                <action android:name="com.google.firebase.MESSAGING_EVENT" />
            </intent-filter>
        </service>
        
        <!-- FileProvider for sharing files -->
        <provider
            android:name="androidx.core.content.FileProvider"
            android:authorities="${applicationId}.fileprovider"
            android:exported="false"
            android:grantUriPermissions="true">
            <meta-data
                android:name="android.support.FILE_PROVIDER_PATHS"
                android:resource="@xml/file_paths" />
        </provider>
        
        <!-- Flutter plugin registration (auto-generated, don't modify manually) -->
        <meta-data
            android:name="flutterEmbedding"
            android:value="2" />
    </application>
</manifest>
```

**Explanation:**

- **Permission declarations**: All permissions must be declared before the `<application>` tag. Android 6.0+ (API 23+) requires runtime permission requests in addition to manifest declarations.
- **SDK-specific permissions**: Use `android:maxSdkVersion` to limit permissions to older Android versions when newer versions use different permission models (e.g., storage permissions changed in API 29 and 33).
- **Hardware features**: `<uses-feature>` declares hardware requirements. Set `android:required="false"` if your app works without the feature (e.g., camera for QR scanning but not essential).
- **Application attributes**:
  - `android:allowBackup="false"`: Prevents auto-backup to Google Drive (security best practice for sensitive apps).
  - `android:networkSecurityConfig`: References XML file for certificate pinning or custom trust anchors.
  - `android:usesCleartextTraffic="false"`: Enforces HTTPS (disable only for development with local servers).
- **Activity configuration**:
  - `android:launchMode="singleTop"`: Standard Flutter behavior; prevents multiple activity instances.
  - `android:configChanges`: Prevents activity recreation on rotation/locale changes (Flutter handles these internally).
  - `android:windowSoftInputMode="adjustResize"`: Ensures keyboard appearance resizes the view rather than covering it.
- **Intent filters**:
  - Standard launcher intent for app icon.
  - Deep linking with `android:autoVerify="true"` enables Android App Links (verified domain association).
  - Custom URL schemes (`myapp://`) for app-to-app communication.

### **Gradle Configuration**

Gradle files control the Android build process, dependencies, and signing configuration.

```gradle
// android/build.gradle (Project level)
// This file configures the build environment for all modules

buildscript {
    ext.kotlin_version = '1.9.0'
    // Kotlin version must be compatible with Flutter and plugins
    
    repositories {
        google()  // Google's Maven repository
        mavenCentral()  // Central Maven repository
        // Avoid jcenter() - deprecated and read-only
    }

    dependencies {
        // Android Gradle Plugin version
        // Must be compatible with your Gradle wrapper version
        classpath 'com.android.tools.build:gradle:8.1.0'
        
        // Kotlin Gradle plugin
        classpath "org.jetbrains.kotlin:kotlin-gradle-plugin:$kotlin_version"
        
        // Google Services plugin (for Firebase)
        classpath 'com.google.gms:google-services:4.3.15'
        
        // Crashlytics plugin (optional)
        classpath 'com.google.firebase:firebase-crashlytics-gradle:2.9.9'
    }
}

allprojects {
    repositories {
        google()
        mavenCentral()
        
        // Private repository example
        maven {
            url "https://maven.mycompany.com/repository"
            credentials {
                username = System.getenv("MAVEN_USERNAME")
                password = System.getenv("MAVEN_PASSWORD")
            }
        }
    }
    
    // Configure all subprojects
    subprojects {
        project.evaluationDependsOn(':app')
        // Ensures app module is evaluated first
    }
}

// Clean task
tasks.register("clean", Delete) {
    delete rootProject.buildDir
}
```

```gradle
// android/app/build.gradle (App level)
// This file configures your specific application module

plugins {
    id "com.android.application"
    id "kotlin-android"
    id "dev.flutter.flutter-gradle-plugin"
    id "com.google.gms.google-services"  // Apply Firebase plugin
}

def localProperties = new Properties()
def localPropertiesFile = rootProject.file('local.properties')
if (localPropertiesFile.exists()) {
    localPropertiesFile.withReader('UTF-8') { reader ->
        localProperties.load(reader)
    }
}

def flutterVersionCode = localProperties.getProperty('flutter.versionCode')
if (flutterVersionCode == null) {
    flutterVersionCode = '1'
}

def flutterVersionName = localProperties.getProperty('flutter.versionName')
if (flutterVersionName == null) {
    flutterVersionName = '1.0'
}

android {
    namespace "com.example.myapp"
    // Namespace replaces the old package attribute in AndroidManifest.xml
    // Must match applicationId
    
    compileSdkVersion 34
    // Target the latest stable SDK version
    
    ndkVersion flutter.ndkVersion
    // Use Flutter's recommended NDK version for native code

    compileOptions {
        sourceCompatibility JavaVersion.VERSION_17
        targetCompatibility JavaVersion.VERSION_17
        // Java 17 is recommended for Flutter 3.10+
        
        coreLibraryDesugaringEnabled true
        // Required for some plugins using Java 8+ APIs on older Android versions
    }

    kotlinOptions {
        jvmTarget = '17'
        // Match Java version
    }

    sourceSets {
        main.java.srcDirs += 'src/main/kotlin'
    }

    defaultConfig {
        applicationId "com.example.myapp"
        // Unique app identifier (cannot be changed after first release)
        
        minSdkVersion 21
        // Minimum Android 5.0 (API 21) - Flutter's minimum recommendation
        // Increase if using APIs from higher versions
        
        targetSdkVersion 34
        // Must match compileSdkVersion
        // Update when new Android versions release
        
        versionCode flutterVersionCode.toInteger()
        // Integer version for Play Store (must increment for each release)
        
        versionName flutterVersionName
        // User-visible version string
        
        multiDexEnabled true
        // Required if method count exceeds 64k (common with many plugins)
        
        // ABI filters (optional - reduces APK size by excluding architectures)
        ndk {
            abiFilters 'armeabi-v7a', 'arm64-v8a', 'x86_64'
            // Exclude 'x86' (emulator only) for production unless needed
        }
    }

    signingConfigs {
        release {
            // Load signing configuration from environment variables or properties
            // NEVER commit keystore passwords to version control
            
            if (System.getenv("KEYSTORE_PATH")) {
                storeFile file(System.getenv("KEYSTORE_PATH"))
                storePassword System.getenv("KEYSTORE_PASSWORD")
                keyAlias System.getenv("KEY_ALIAS")
                keyPassword System.getenv("KEY_PASSWORD")
            } else {
                // Fallback to local.properties for development
                keyAlias localProperties.getProperty('key.alias')
                keyPassword localProperties.getProperty('key.password')
                storeFile localProperties.getProperty('key.store') ? file(localProperties.getProperty('key.store')) : null
                storePassword localProperties.getProperty('key.store.password')
            }
        }
        
        debug {
            storeFile file('debug.keystore')
            storePassword 'android'
            keyAlias 'androiddebugkey'
            keyPassword 'android'
        }
    }

    buildTypes {
        release {
            signingConfig signingConfigs.release
            
            // Code shrinking and obfuscation
            minifyEnabled true
            shrinkResources true
            // Removes unused code and resources
            
            proguardFiles getDefaultProguardFile('proguard-android.txt'), 'proguard-rules.pro'
            // ProGuard rules for code shrinking
            
            // Additional optimizations
            crunchPngs true
            // Compresses PNG assets during build
        }
        
        debug {
            signingConfig signingConfigs.debug
            minifyEnabled false
            shrinkResources false
            // Disable shrinking for faster debug builds
        }
        
        // Custom build type for staging
        staging {
            initWith release
            signingConfig signingConfigs.debug
            applicationIdSuffix ".staging"
            versionNameSuffix "-staging"
            // Creates separate app ID for testing (e.g., com.example.myapp.staging)
        }
    }
    
    // Flavor dimensions (product variants)
    flavorDimensions "environment"
    
    productFlavors {
        development {
            dimension "environment"
            applicationIdSuffix ".dev"
            versionNameSuffix "-dev"
            resValue "string", "app_name", "MyApp Dev"
            // Different app name for development builds
        }
        
        production {
            dimension "environment"
            resValue "string", "app_name", "MyApp"
        }
    }
    
    // Build configuration for different CPU architectures
    splits {
        abi {
            enable true
            reset()
            include 'armeabi-v7a', 'arm64-v8a', 'x86_64'
            universalApk false
            // Generate separate APKs per architecture (smaller sizes)
            // Set universalApk true for single APK supporting all architectures
        }
    }
    
    // Dependency resolution strategy
    configurations.all {
        resolutionStrategy {
            // Force specific versions to resolve conflicts
            force 'org.jetbrains.kotlin:kotlin-stdlib:1.9.0'
        }
    }
}

flutter {
    source '../..'
    // Points to Flutter SDK source
}

dependencies {
    // Core desugaring library (required for API 26+ features on older devices)
    coreLibraryDesugaring 'com.android.tools:desugar_jdk_libs:2.0.3'
    
    // Kotlin standard library (usually included by Flutter plugins)
    implementation "org.jetbrains.kotlin:kotlin-stdlib-jdk8:$kotlin_version"
    
    // Security library (for encrypted SharedPreferences)
    implementation 'androidx.security:security-crypto:1.1.0-alpha06'
    
    // Lifecycle components (for background processing)
    implementation 'androidx.lifecycle:lifecycle-runtime-ktx:2.6.2'
    
    // WorkManager (for background tasks)
    implementation 'androidx.work:work-runtime-ktx:2.8.1'
}
```

**Explanation:**

- **Project-level vs. App-level**: Project-level `build.gradle` configures the build environment; app-level configures your specific application.
- **SDK versions**:
  - `minSdkVersion`: Minimum Android version your app supports (21 = Android 5.0).
  - `targetSdkVersion`: Android version your app is tested against (should be latest).
  - `compileSdkVersion`: SDK version used to compile your app (should match target).
- **Signing configuration**: Release builds must be signed with a keystore. Use environment variables in CI/CD; use `local.properties` for local development (add to `.gitignore`).
- **Build types**:
  - `debug`: Unsigned, unoptimized, includes debug symbols.
  - `release`: Signed, obfuscated, shrunk, optimized.
  - Custom types (e.g., `staging`) inherit from existing types with modifications.
- **Product flavors**: Create different app variants (dev/prod) with different configurations, resources, and app IDs.
- **ProGuard/R8**: `minifyEnabled true` enables code shrinking. `shrinkResources true` removes unused resources. Both reduce APK size and improve security through obfuscation.
- **ABI splits**: Building separate APKs for different CPU architectures (`armeabi-v7a`, `arm64-v8a`, `x86_64`) reduces download size by ~40% per architecture.

### **ProGuard Rules**

ProGuard (R8) obfuscates and shrinks your code. Flutter requires specific rules to prevent obfuscation of framework code.

```proguard
# android/app/proguard-rules.pro

# Flutter-specific rules
-keep class io.flutter.app.** { *; }
-keep class io.flutter.plugin.** { *; }
-keep class io.flutter.util.** { *; }
-keep class io.flutter.view.** { *; }
-keep class io.flutter.** { *; }
-keep class io.flutter.plugins.** { *; }
-keep class com.google.firebase.** { *; }  # Firebase
-keep class androidx.lifecycle.** { *; }   # Lifecycle components

# Keep native methods
-keepclasseswithmembernames class * {
    native <methods>;
}

# Keep custom application classes
-keep class com.example.myapp.** { *; }
-keepclassmembers class com.example.myapp.** {
    *;
}

# JSON serialization (if using reflection)
-keepclassmembers class * {
    @com.google.gson.annotations.SerializedName <fields>;
}

# Prevent obfuscation of model classes (if using JSON parsing)
-keep class com.example.myapp.models.** { *; }
-keepclassmembers class com.example.myapp.models.** {
    <fields>;
    <init>();
}

# Network (Dio/Retrofit)
-dontwarn okhttp3.**
-dontwarn okio.**
-dontwarn javax.annotation.**
-keepnames class okhttp3.internal.publicsynchronized.PublicSynchronizedPool

# Database (Room/SQLite)
-keep class * extends androidx.room.RoomDatabase
-dontwarn androidx.room.paging.**

# Crashlytics
-keepattributes *Annotation*
-keepattributes SourceFile,LineNumberTable
-keep public class * extends java.lang.Exception

# Remove logging in release builds
-assumenosideeffects class android.util.Log {
    public static boolean isLoggable(java.lang.String, int);
    public static int v(...);
    public static int d(...);
    public static int i(...);
    public static int w(...);
    public static int e(...);
}
```

**Explanation:**

- **Keep rules**: `-keep` prevents classes from being renamed or removed. Essential for classes accessed via reflection (Flutter plugins, JSON serialization).
- **Flutter framework**: The Flutter engine uses JNI (Java Native Interface) to communicate with Android. Obfuscating these classes breaks the connection.
- **Model classes**: If you parse JSON into Dart objects via platform channels, the Java/Kotlin model classes must not be obfuscated.
- **Logging removal**: `-assumenosideeffects` removes Log calls in release builds for security (prevents leaking sensitive data in logs).
- **Troubleshooting**: If you get `ClassNotFoundException` or `NoSuchMethodError` in release builds only, you likely need additional `-keep` rules.

---

## **51.2 iOS Configuration**

iOS configuration involves Xcode project files, Info.plist, and Podfile management. These files reside in the `ios/` directory.

### **Info.plist Configuration**

`Info.plist` (Property List) is the iOS equivalent of AndroidManifest.xml, containing app metadata, permissions, and configuration.

```xml
<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE plist PUBLIC "-//Apple//DTD PLIST 1.0//EN" "http://www.apple.com/DTDs/PropertyList-1.0.dtd">
<plist version="1.0">
<dict>
    <!-- Bundle identifier and version -->
    <key>CFBundleIdentifier</key>
    <string>$(PRODUCT_BUNDLE_IDENTIFIER)</string>
    <key>CFBundleVersion</key>
    <string>$(CURRENT_PROJECT_VERSION)</string>
    <key>CFBundleShortVersionString</key>
    <string>$(MARKETING_VERSION)</string>
    
    <!-- App name displayed on home screen -->
    <key>CFBundleDisplayName</key>
    <string>$(APP_DISPLAY_NAME)</string>
    
    <!-- Required for Flutter -->
    <key>CFBundleExecutable</key>
    <string>$(EXECUTABLE_NAME)</string>
    <key>CFBundlePackageType</key>
    <string>APPL</string>
    
    <!-- UI configuration -->
    <key>UISupportedInterfaceOrientations</key>
    <array>
        <string>UIInterfaceOrientationPortrait</string>
        <string>UIInterfaceOrientationLandscapeLeft</string>
        <string>UIInterfaceOrientationLandscapeRight</string>
    </array>
    <key>UISupportedInterfaceOrientations~ipad</key>
    <array>
        <string>UIInterfaceOrientationPortrait</string>
        <string>UIInterfaceOrientationPortraitUpsideDown</string>
        <string>UIInterfaceOrientationLandscapeLeft</string>
        <string>UIInterfaceOrientationLandscapeRight</string>
    </array>
    
    <!-- Status bar style -->
    <key>UIViewControllerBasedStatusBarAppearance</key>
    <false/>
    <key>UIStatusBarHidden</key>
    <false/>
    
    <!-- Required device capabilities -->
    <key>UIRequiredDeviceCapabilities</key>
    <array>
        <string>armv7</string>
        <string>gps</string>  <!-- Optional: if app requires GPS -->
    </array>
    
    <!-- App Transport Security (ATS) - HTTPS enforcement -->
    <key>NSAppTransportSecurity</key>
    <dict>
        <key>NSAllowsArbitraryLoads</key>
        <false/>
        <!-- Allow specific domains for development only -->
        <key>NSExceptionDomains</key>
        <dict>
            <key>localhost</key>
            <dict>
                <key>NSExceptionAllowsInsecureHTTPLoads</key>
                <true/>
                <key>NSIncludesSubdomains</key>
                <true/>
            </dict>
        </dict>
    </dict>
    
    <!-- PERMISSION DESCRIPTIONS (Privacy - * Usage Description) -->
    <!-- These strings appear in the permission dialog -->
    <key>NSCameraUsageDescription</key>
    <string>This app needs camera access to scan QR codes and take profile photos.</string>
    
    <key>NSPhotoLibraryUsageDescription</key>
    <string>This app needs access to photos to allow you to select profile pictures.</string>
    
    <key>NSPhotoLibraryAddUsageDescription</key>
    <string>This app needs permission to save photos to your library.</string>
    
    <key>NSLocationWhenInUseUsageDescription</key>
    <string>This app needs your location to show nearby stores.</string>
    
    <key>NSLocationAlwaysUsageDescription</key>
    <string>This app needs background location to track your delivery.</string>
    
    <key>NSLocationAlwaysAndWhenInUseUsageDescription</key>
    <string>This app needs your location to provide navigation features.</string>
    
    <key>NSBluetoothAlwaysUsageDescription</key>
    <string>This app uses Bluetooth to connect to peripheral devices.</string>
    
    <key>NSMicrophoneUsageDescription</key>
    <string>This app needs microphone access for voice messages.</string>
    
    <key>NSFaceIDUsageDescription</key>
    <string>This app uses Face ID to secure your data.</string>
    
    <key>NSUserTrackingUsageDescription</key>
    <string>This app tracks usage data to improve your experience.</string>
    
    <!-- Background modes -->
    <key>UIBackgroundModes</key>
    <array>
        <string>fetch</string>
        <string>remote-notification</string>
        <string>location</string>
        <string>processing</string>
    </array>
    
    <!-- URL Schemes (Deep Linking) -->
    <key>CFBundleURLTypes</key>
    <array>
        <dict>
            <key>CFBundleURLName</key>
            <string>com.example.myapp</string>
            <key>CFBundleURLSchemes</key>
            <array>
                <string>myapp</string>
            </array>
        </dict>
    </array>
    
    <!-- Universal Links (Associated Domains) -->
    <key>com.apple.developer.associated-domains</key>
    <array>
        <string>applinks:example.com</string>
        <string>webcredentials:example.com</string>
    </array>
    
    <!-- Launch screen -->
    <key>UILaunchStoryboardName</key>
    <string>LaunchScreen</string>
    
    <!-- Main storyboard (usually not used in Flutter) -->
    <key>UIMainStoryboardFile</key>
    <string>Main</string>
    
    <!-- Flutter-specific -->
    <key>FlutterDeepLinkingEnabled</key>
    <true/>
    
    <!-- Disable hardware keyboard detection (iPad) -->
    <key>UIRequiresPersistentWiFi</key>
    <true/>
</dict>
</plist>
```

**Explanation:**

- **Bundle identifier**: `CFBundleIdentifier` must match your App ID in Apple Developer Portal. Use variables (`$(PRODUCT_BUNDLE_IDENTIFIER)`) to allow Xcode to manage this.
- **Privacy descriptions**: iOS 10+ requires descriptive strings for every permission. These appear in the system permission dialog. Be specific about why you need the permission—vague descriptions lead to App Store rejection.
- **App Transport Security (ATS)**: iOS 9+ requires HTTPS by default. The `NSExceptionAllowsInsecureHTTPLoads` key allows HTTP for specific domains (e.g., `localhost` for development). Remove exceptions before App Store submission.
- **Background modes**: Declare what your app does in the background. Each mode requires justification during App Store review.
  - `fetch`: Background app refresh.
  - `remote-notification`: Push notifications with background processing.
  - `location`: Background location updates.
  - `processing`: Background tasks (iOS 13+).
- **URL Schemes**: Custom schemes (`myapp://`) for app launching. Register the scheme in `CFBundleURLTypes`.
- **Associated Domains**: Required for Universal Links (iOS equivalent to Android App Links). Must match the entitlements file and your domain's `.well-known/apple-app-site-association` file.

### **Podfile Configuration**

CocoaPods manages iOS dependencies. The `Podfile` configures how pods (libraries) are integrated.

```ruby
# ios/Podfile
# Uncomment this line to define a global platform for your project
platform :ios, '13.0'
# Minimum iOS version - Flutter 3.10+ recommends iOS 13+
# Higher minimum versions allow using modern iOS APIs

# CocoaPods analytics sends network stats synchronously
# Disable for privacy
ENV['COCOAPODS_DISABLE_STATS'] = 'true'

project 'Runner', {
  'Debug' => :debug,
  'Profile' => :release,
  'Release' => :release,
}

def flutter_root
  generated_xcode_build_settings_path = File.expand_path(File.join('..', '..', 'Flutter', 'Generated.xcconfig'), __FILE__)
  unless File.exist?(generated_xcode_build_settings_path)
    raise "#{generated_xcode_build_settings_path} must exist. If you're running pod install manually, make sure flutter pub get is executed first"
  end

  File.foreach(generated_xcode_build_settings_path) do |line|
    matches = line.match(/FLUTTER_ROOT\=(.*)/)
    return matches[1].strip if matches
  end
  raise "Missing FLUTTER_ROOT in #{generated_xcode_build_settings_path}. Try deleting Generated.xcconfig, then run flutter pub get"
end

require File.expand_path(File.join('packages', 'flutter_tools', 'bin', 'podhelper'), flutter_root)

flutter_ios_podfile_setup

target 'Runner' do
  use_frameworks!
  use_modular_headers!
  # Required for Swift-based plugins
  
  # Flutter pods
  flutter_install_all_ios_pods File.dirname(File.realpath(__FILE__))
  
  # Specific pod versions (if needed to resolve conflicts)
  # pod 'FirebaseCore', '10.15.0'
  
  target 'RunnerTests' do
    inherit! :search_paths
    # Inherits pods from parent target
  end
end

# Post-install configuration
post_install do |installer|
  installer.pods_project.targets.each do |target|
    flutter_additional_ios_build_settings(target)
    
    # Additional build settings for all pods
    target.build_configurations.each do |config|
      config.build_settings['IPHONEOS_DEPLOYMENT_TARGET'] = '13.0'
      # Ensure all pods use minimum iOS 13.0
      
      config.build_settings['ENABLE_BITCODE'] = 'NO'
      # Bitcode is deprecated in Xcode 14+
      
      config.build_settings['BUILD_LIBRARY_FOR_DISTRIBUTION'] = 'YES'
      # Required for XCFramework distribution
      
      # Architecture settings
      config.build_settings['EXCLUDED_ARCHS[sdk=iphonesimulator*]'] = 'arm64'
      # Exclude arm64 for simulators on Apple Silicon Macs if needed
      
      # Swift version for pods with Swift code
      config.build_settings['SWIFT_VERSION'] = '5.0'
      
      # Code signing settings for CI/CD
      config.build_settings['CODE_SIGNING_ALLOWED'] = 'NO'
      config.build_settings['CODE_SIGNING_REQUIRED'] = 'NO'
      # Disable automatic code signing in pods (handled by main target)
    end
  end
end
```

**Explanation:**

- **Platform version**: `platform :ios, '13.0'` sets the minimum iOS version. This must match your Xcode deployment target and Info.plist.
- **use_frameworks!**: Required for Swift-based plugins. Flutter plugins use dynamic frameworks.
- **flutter_install_all_ios_pods**: Automatically adds Flutter plugin dependencies.
- **Post-install hook**: Configures build settings for all pods after installation.
  - `IPHONEOS_DEPLOYMENT_TARGET`: Ensures all pods match your minimum version.
  - `ENABLE_BITCODE`: Must be `NO` for Flutter (Xcode 14 deprecated Bitcode).
  - `EXCLUDED_ARCHS`: Sometimes needed to fix simulator builds on Apple Silicon Macs (M1/M2).
- **Pod installation workflow**:
  1. Modify `Podfile` if needed.
  2. Run `flutter pub get` to update Flutter dependencies.
  3. Run `cd ios && pod install --repo-update` to install/update pods.
  4. Open `Runner.xcworkspace` (not `.xcodeproj`) in Xcode.

---

## **51.3 Platform-Specific Permissions**

Handling runtime permissions requires platform-specific code, though Flutter plugins abstract most of it. Understanding the native side is crucial for debugging.

### **Android Runtime Permissions**

Android 6.0+ (API 23) requires requesting permissions at runtime in addition to manifest declarations.

```kotlin
// android/app/src/main/kotlin/com/example/myapp/MainActivity.kt
package com.example.myapp

import android.Manifest
import android.content.pm.PackageManager
import android.os.Build
import android.os.Bundle
import androidx.core.app.ActivityCompat
import androidx.core.content.ContextCompat
import io.flutter.embedding.android.FlutterActivity
import io.flutter.embedding.engine.FlutterEngine
import io.flutter.plugin.common.MethodChannel

class MainActivity: FlutterActivity() {
    private val CHANNEL = "com.example.myapp/permissions"
    private val PERMISSION_REQUEST_CODE = 1001
    
    override fun configureFlutterEngine(flutterEngine: FlutterEngine) {
        super.configureFlutterEngine(flutterEngine)
        
        MethodChannel(flutterEngine.dartExecutor.binaryMessenger, CHANNEL).setMethodCallHandler {
            call, result ->
            when (call.method) {
                "checkPermission" -> {
                    val permission = call.argument<String>("permission")
                    result.success(checkPermission(permission))
                }
                "requestPermission" -> {
                    val permission = call.argument<String>("permission")
                    requestPermission(permission, result)
                }
                else -> result.notImplemented()
            }
        }
    }
    
    private fun checkPermission(permission: String?): Boolean {
        return if (Build.VERSION.SDK_INT >= Build.VERSION_CODES.M) {
            ContextCompat.checkSelfPermission(this, permission!!) == 
                PackageManager.PERMISSION_GRANTED
        } else {
            true // Permissions automatically granted below API 23
        }
    }
    
    private fun requestPermission(permission: String?, result: MethodChannel.Result) {
        if (Build.VERSION.SDK_INT >= Build.VERSION_CODES.M) {
            ActivityCompat.requestPermissions(
                this,
                arrayOf(permission),
                PERMISSION_REQUEST_CODE
            )
            // Note: In production, store the result callback and handle 
            // onRequestPermissionsResult
        }
        result.success(true)
    }
    
    // Handle permission result
    override fun onRequestPermissionsResult(
        requestCode: Int,
        permissions: Array<out String>,
        grantResults: IntArray
    ) {
        super.onRequestPermissionsResult(requestCode, permissions, grantResults)
        // Notify Flutter about the result via EventChannel or MethodChannel
    }
}
```

**Explanation:**

- **MethodChannel**: Communication bridge between Dart and native Android (Kotlin/Java). The channel name must match on both sides.
- **Permission check**: `ContextCompat.checkSelfPermission()` returns `PERMISSION_GRANTED` or `PERMISSION_DENIED`.
- **Request flow**: 
  1. Check if permission is already granted.
  2. If not, call `requestPermissions()` which shows the system dialog.
  3. Handle the result in `onRequestPermissionsResult()`.
- **API level checks**: Runtime permissions only required for API 23+ (Android 6.0). Lower versions grant at install time.

### **iOS Permission Handling**

iOS handles permissions through the Info.plist descriptions and native APIs, but the system manages the dialog presentation.

```swift
// ios/Runner/AppDelegate.swift
import UIKit
import Flutter
import AVFoundation  // For camera/microphone
import Photos      // For photo library
import CoreLocation // For location

@UIApplicationMain
@objc class AppDelegate: FlutterAppDelegate {
    override func application(
        _ application: UIApplication,
        didFinishLaunchingWithOptions launchOptions: [UIApplication.LaunchOptionsKey: Any]?
    ) -> Bool {
        let controller = window?.rootViewController as! FlutterViewController
        let permissionChannel = FlutterMethodChannel(
            name: "com.example.myapp/permissions",
            binaryMessenger: controller.binaryMessenger
        )
        
        permissionChannel.setMethodCallHandler { [weak self] (call, result) in
            switch call.method {
            case "checkCameraPermission":
                self?.checkCameraPermission(result: result)
            case "requestCameraPermission":
                self?.requestCameraPermission(result: result)
            case "checkPhotoLibraryPermission":
                self?.checkPhotoLibraryPermission(result: result)
            default:
                result(FlutterMethodNotImplemented)
            }
        }
        
        GeneratedPluginRegistrant.register(with: self)
        return super.application(application, didFinishLaunchingWithOptions: launchOptions)
    }
    
    private func checkCameraPermission(result: @escaping FlutterResult) {
        let status = AVCaptureDevice.authorizationStatus(for: .video)
        switch status {
        case .authorized:
            result(true)
        case .denied, .restricted:
            result(false)
        case .notDetermined:
            result(nil) // Permission not requested yet
        @unknown default:
            result(false)
        }
    }
    
    private func requestCameraPermission(result: @escaping FlutterResult) {
        AVCaptureDevice.requestAccess(for: .video) { granted in
            DispatchQueue.main.async {
                result(granted)
            }
        }
    }
    
    private func checkPhotoLibraryPermission(result: @escaping FlutterResult) {
        let status = PHPhotoLibrary.authorizationStatus()
        result(status == .authorized)
    }
}
```

**Explanation:**

- **Swift MethodChannel**: Similar to Android, but uses Swift syntax and iOS APIs.
- **Authorization status**: iOS has three states: `.notDetermined` (never asked), `.restricted` (parental controls), `.denied` (user said no), `.authorized` (granted).
- **Async handling**: iOS permission requests are asynchronous. Use completion handlers and dispatch results to main queue for Flutter.
- **Privacy compliance**: iOS 14+ requires `NSPhotoLibraryUsageDescription` for photo access, and iOS 15+ has stricter rules for "Allow Once" permissions.

---

## **51.4 Build Troubleshooting**

Platform-specific build errors are common in Flutter. Here are solutions to frequent issues.

### **Android Build Issues**

```gradle
// Common fixes in android/build.gradle

// 1. Dependency resolution conflicts
allprojects {
    configurations.all {
        resolutionStrategy {
            // Force specific versions to resolve conflicts
            force 'com.google.code.gson:gson:2.10.1'
            force 'org.jetbrains.kotlin:kotlin-stdlib:1.9.0'
            
            // Cache dynamic versions for 10 minutes
            cacheDynamicVersionsFor 10, 'minutes'
            cacheChangingModulesFor 4, 'hours'
        }
    }
}

// 2. Duplicate classes/Dex merger errors
android {
    packagingOptions {
        pickFirst '**/libc++_shared.so'
        pickFirst '**/libjsc.so'
        exclude 'META-INF/DEPENDENCIES'
        exclude 'META-INF/LICENSE'
    }
}

// 3. Out of memory errors
org.gradle.jvmargs=-Xmx4096m -XX:MaxMetaspaceSize=512m -XX:+HeapDumpOnOutOfMemoryError
// In gradle.properties
```

### **iOS Build Issues**

```ruby
# Common fixes in Podfile

# 1. Architecture issues on Apple Silicon Macs
post_install do |installer|
  installer.pods_project.build_configurations.each do |config|
    config.build_settings["EXCLUDED_ARCHS[sdk=iphonesimulator*]"] = "arm64"
    # Or for physical device issues:
    # config.build_settings['ONLY_ACTIVE_ARCH'] = 'YES'
  end
end

# 2. Swift version mismatches
post_install do |installer|
  installer.pods_project.targets.each do |target|
    target.build_configurations.each do |config|
      if config.build_settings['SWIFT_VERSION'].nil?
        config.build_settings['SWIFT_VERSION'] = '5.0'
      end
    end
  end
end

# 3. Module map issues
# In Podfile, add to target:
pod 'FirebaseCore', :modular_headers => true
```

**Explanation:**

- **Duplicate classes**: Occurs when two dependencies include the same library (e.g., two versions of Gson). Use `force` or `exclude` in Gradle.
- **Dex merger**: 64k method limit exceeded. Enable `multiDexEnabled true` in `defaultConfig`.
- **iOS architecture**: Apple Silicon Macs (M1/M2) use `arm64` for simulators, but some pods expect `x86_64`. The `EXCLUDED_ARCHS` fix forces Rosetta translation.
- **CocoaPods cache**: If pods behave strangely:
  ```bash
  cd ios
  rm -rf Pods Podfile.lock
  pod deintegrate
  pod cache clean --all
  pod install --repo-update
  ```

---

## **Chapter Summary**

In this chapter, we covered platform-specific configuration essential for production Flutter deployment:

### **Key Takeaways:**

1. **Android Configuration**:
   - **AndroidManifest.xml**: Permissions, hardware features, deep links, and application metadata.
   - **Gradle**: SDK versions (min/target/compile), signing configs, build types (debug/release/staging), product flavors (dev/prod), and ProGuard rules.
   - **ProGuard/R8**: Keep rules for Flutter framework, JSON models, and native methods. Remove logging in production.

2. **iOS Configuration**:
   - **Info.plist**: Bundle ID, version, privacy descriptions (critical for App Store), ATS settings, background modes, and URL schemes.
   - **Podfile**: Minimum iOS version, framework usage, post-install hooks for build settings (Bitcode disabled, deployment targets).
   - **Xcode**: Build using `.xcworkspace` not `.xcodeproj`, manage signing via Xcode or CI environment variables.

3. **Permissions**:
   - **Android**: Manifest declarations + runtime requests for API 23+. Use `ContextCompat` for compatibility.
   - **iOS**: Info.plist descriptions mandatory. Check authorization status before requesting. Handle async callbacks on main thread.

4. **Security**:
   - Android: Disable `allowBackup`, use `networkSecurityConfig`, enforce HTTPS (ATS).
   - iOS: Strict ATS settings, secure keychain usage, privacy descriptions must be specific (App Store rejection risk).

5. **Troubleshooting**:
   - Gradle dependency conflicts: Use `resolutionStrategy` and `force`.
   - CocoaPods issues: Clean cache, update repos, check architecture settings for Apple Silicon.
   - Version alignment: Ensure all platform configs match (e.g., iOS 13.0 in Podfile, Xcode, and Info.plist).

### **Platform Checklist:**

- [ ] Android: `minSdkVersion` appropriate for target devices (21+ recommended)
- [ ] Android: `targetSdkVersion` at latest stable (34)
- [ ] Android: ProGuard rules for all plugins and models
- [ ] Android: Signing config for release (environment variables)
- [ ] iOS: Info.plist privacy descriptions for all permissions
- [ ] iOS: `Podfile` platform version matches Xcode project
- [ ] iOS: `ENABLE_BITCODE` set to `NO` (Xcode 14+)
- [ ] Both: Deep linking/Universal Links configured
- [ ] Both: Background modes justified and declared
- [ ] Both: App icons and launch screens configured

### **Next Steps:**

The next chapter (Chapter 52) will cover **Migration Guides**, detailing:
- AndroidX migration strategies
- Null safety migration for legacy codebases
- Material 3 (Material You) migration
- Deprecated API replacements and breaking changes
- Automated migration tools and scripts

---

**End of Chapter 51**

---

# **Next Chapter: Chapter 52 - Migration Guides**

Chapter 52 will provide comprehensive strategies for migrating existing Flutter applications to modern standards, including AndroidX compatibility, null safety adoption, Material 3 theming, and handling breaking changes in major dependency updates.



<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='50. package_ecosystem_guide.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='52. migration_guides.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
